# Proyecto 2
Estuardo Díaz 16110

In [19]:
import numpy as np
from numpy import linalg
import scipy

### Factorización $LU$ de matrices

In [20]:
def factorizar(A):
    PL, U = scipy.linalg.lu(A,True)
    return PL, U

## a) Matriz mal condicionada

Definimos la **condición** de $A$ como $cond(A) := ||A||\cdot ||A^{-1}||$

In [15]:
def cond(A):
    inv_A = linalg.inv(A)
    return linalg.norm(A)*linalg.norm(inv_A)

Consideremos la matriz $A$ mal condicionada siguiente:

In [16]:
A = np.array([[2.0,4.0,5.0],[6.0,9.0,8.0],[4.1,5.0,3.0]])
print("A:\n",A)

A:
 [[2.  4.  5. ]
 [6.  9.  8. ]
 [4.1 5.  3. ]]


In [17]:
print("cond(A): ",cond(A))

cond(A):  453.723201467921


Como la condición de $A$ es muy grande, sabemos que la matriz es mal condicionada.

In [21]:
L, U = factorizar(A)

AttributeError: module 'scipy' has no attribute 'linalg'

## b) Matriz de insumo producto

\begin{align}
x &= Ax + y \\
x - Ax &= y \\
x(I - A) &= y\\
x &= (I-A)^{-1}y
\end{align}

In [22]:
A = np.array([[2.0,4.0,5.0],[6.0,9.0,8.0],[4.1,5.0,3.0]])

## c) Regresión múltiple

In [23]:
A = np.array([[2.0,4.0,5.0],[6.0,9.0,8.0],[4.1,5.0,3.0]])